#  Content Based Recommendation System

A content-based recommendation system that suggests similar movies based on their metadata (e.g., genres, keywords, overview). We'll use cosine similarity to measure the similarity between movies by following given steps:
1.	Combine relevant features that you believe will contribute to the similarity between movies. For simplicity, let's use genres, keywords, cast, and overview.
2.	For text features, preprocess the data by converting to lowercase, removing stopwords, and applying stemming or lemmatization if necessary.
3.	Use TfidfVectorizer from sklearn.feature_extraction.text to convert the text data into a matrix of TF-IDF features.
4.	Calculate the cosine similarity between all movies based on the TF-IDF matrix. This will give us a similarity matrix where each element represents the similarity score between a pair of movies.
5.	Write a function that takes a movie title as input, finds that movie's index in the dataset, and returns a list of movies sorted by their similarity score in descending order.

In [166]:
import pandas as pd
import ast

In [167]:
df=pd.read_csv("tmdb_5000_movies.csv")

In [168]:
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [169]:
df=df[['original_title','genres','keywords','overview']]
df=df.rename(columns={'original_title':'title'})
df.head()

,title,genres,keywords,overview
0,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
2,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
3,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
4,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."


In [170]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L

In [171]:
df['genres']=df['genres'].apply(convert)
df['keywords']=df['keywords'].apply(convert)

In [172]:
df.head()

,title,genres,keywords,overview
0,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di..."
1,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha..."
2,Spectre,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...
3,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...
4,John Carter,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca..."


In [173]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def combine_tags(row):
    return row['genres']+row['keywords']

def remove_stopwords(text):
    text=str(text)
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)
    



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\uzair\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [174]:
df['tags']=df.apply(combine_tags, axis=1)

In [175]:
df['overview']=df['overview'].apply(remove_stopwords)

In [176]:
df.head()

,title,genres,keywords,overview,tags
0,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","22nd century, paraplegic Marine dispatched moo...","[Action, Adventure, Fantasy, Science Fiction, ..."
1,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed dead, come bac...","[Adventure, Fantasy, Action, ocean, drug abuse..."
2,Spectre,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",cryptic message Bond’s past sends trail uncove...,"[Action, Adventure, Crime, spy, based on novel..."
3,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","Following death District Attorney Harvey Dent,...","[Action, Crime, Drama, Thriller, dc comics, cr..."
4,John Carter,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","John Carter war-weary, former military captain...","[Action, Adventure, Science Fiction, based on ..."


In [177]:
df['tags']=df['tags'].apply(lambda x: ' '.join(x))

In [178]:
df['tags'].head()

0    Action Adventure Fantasy Science Fiction cultu...
1    Adventure Fantasy Action ocean drug abuse exot...
2    Action Adventure Crime spy based on novel secr...
3    Action Crime Drama Thriller dc comics crime fi...
4    Action Adventure Science Fiction based on nove...
Name: tags, dtype: object

In [179]:
df['tags']=df['tags']+df['overview']

In [180]:
df['tags'].head()

0    Action Adventure Fantasy Science Fiction cultu...
1    Adventure Fantasy Action ocean drug abuse exot...
2    Action Adventure Crime spy based on novel secr...
3    Action Crime Drama Thriller dc comics crime fi...
4    Action Adventure Science Fiction based on nove...
Name: tags, dtype: object

In [181]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=2000)

In [182]:
X = tfidf.fit_transform(df['tags'])

In [183]:
X

<4803x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 124412 stored elements in Compressed Sparse Row format>

In [184]:
movie2idx = pd.Series(df.index, index=df['title'])
movie2idx

title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64

In [185]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances


In [190]:
similarity = cosine_similarity(X, X)
similarity


array([[1.        , 0.03229036, 0.0113861 , ..., 0.01422906, 0.00611833,
        0.        ],
       [0.03229036, 1.        , 0.0325986 , ..., 0.03413101, 0.        ,
        0.00973103],
       [0.0113861 , 0.0325986 , 1.        , ..., 0.02015251, 0.        ,
        0.        ],
       ...,
       [0.01422906, 0.03413101, 0.02015251, ..., 1.        , 0.03871513,
        0.06228915],
       [0.00611833, 0.        , 0.        , ..., 0.03871513, 1.        ,
        0.04476233],
       [0.        , 0.00973103, 0.        , ..., 0.06228915, 0.04476233,
        1.        ]])

In [195]:
def recommend(title):
  idx = movie2idx[title]
  if type(idx) == pd.Series:
    idx = idx.iloc[0]
  query = X[idx]
  scores = cosine_similarity(query, X)
  scores = scores.flatten()
  recommended_idx = (-scores).argsort()[1:6]
  return list(zip(df['title'].iloc[recommended_idx],scores[recommended_idx]))

In [202]:
recommend("Avatar")

[('Aliens', 0.47123522545321583),
 ('Spaceballs', 0.43304130858004375),
 ('Moonraker', 0.4293976754654126),
 ('Lost in Space', 0.4258720721934339),
 ('Alien³', 0.42429357821901564)]